In [5]:
import pandas as pd
import numpy as np
import json
import torch

Install spotlight package 

In [2]:
!pip install --user git+https://github.com/maciejkula/spotlight.git@master#egg=spotlight

In [3]:
!pip install --user --upgrade pip

Requirement already up-to-date: pip in /users/msc/2413917c/.local/lib/python3.6/site-packages (19.2.1)


Dataset could be downloaded from here: https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Electronics_v1_00.tsv.gz 

In [4]:
df = pd.read_csv('amazon_reviews_us_Electronics_v1_00.tsv.gz', delimiter='\t',error_bad_lines=False)
df.head()

b'Skipping line 9076: expected 15 fields, saw 22\nSkipping line 19256: expected 15 fields, saw 22\nSkipping line 24313: expected 15 fields, saw 22\nSkipping line 47211: expected 15 fields, saw 22\nSkipping line 54295: expected 15 fields, saw 22\nSkipping line 56641: expected 15 fields, saw 22\nSkipping line 63067: expected 15 fields, saw 22\n'
b'Skipping line 93796: expected 15 fields, saw 22\n'
b'Skipping line 132806: expected 15 fields, saw 22\nSkipping line 164631: expected 15 fields, saw 22\nSkipping line 167019: expected 15 fields, saw 22\nSkipping line 167212: expected 15 fields, saw 22\n'
b'Skipping line 198103: expected 15 fields, saw 22\nSkipping line 199191: expected 15 fields, saw 22\nSkipping line 202841: expected 15 fields, saw 22\nSkipping line 218228: expected 15 fields, saw 22\nSkipping line 235900: expected 15 fields, saw 22\n'
b'Skipping line 277761: expected 15 fields, saw 22\nSkipping line 304582: expected 15 fields, saw 22\nSkipping line 312029: expected 15 fields,

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,41409413,R2MTG1GCZLR2DK,B00428R89M,112201306,yoomall 5M Antenna WIFI RP-SMA Female to Male ...,Electronics,5,0,0,N,Y,Five Stars,As described.,2015-08-31
1,US,49668221,R2HBOEM8LE9928,B000068O48,734576678,"Hosa GPM-103 3.5mm TRS to 1/4"" TRS Adaptor",Electronics,5,0,0,N,Y,It works as advertising.,It works as advertising.,2015-08-31
2,US,12338275,R1P4RW1R9FDPEE,B000GGKOG8,614448099,Channel Master Titan 2 Antenna Preamplifier,Electronics,5,1,1,N,Y,Five Stars,Works pissa,2015-08-31
3,US,38487968,R1EBPM82ENI67M,B000NU4OTA,72265257,LIMTECH Wall charger + USB Hotsync & Charging ...,Electronics,1,0,0,N,Y,One Star,Did not work at all.,2015-08-31
4,US,23732619,R372S58V6D11AT,B00JOQIO6S,308169188,Skullcandy Air Raid Portable Bluetooth Speaker,Electronics,5,1,1,N,Y,Overall pleased with the item,Works well. Bass is somewhat lacking but is pr...,2015-08-31


In [5]:
df1 = df.copy()
df1.count()

marketplace          3091024
customer_id          3091024
review_id            3091024
product_id           3091024
product_parent       3091024
product_title        3091020
product_category     3091024
star_rating          3091024
helpful_votes        3091024
total_votes          3091024
vine                 3091024
verified_purchase    3091024
review_headline      3090993
review_body          3090936
review_date          3091000
dtype: int64

In [ ]:
df1 = df1[df1.review_date.notnull()]
df1 = df1[df1.product_title.notnull()]
df1 = df1[df1.product_id.notnull()]
df1 = df1[df1.customer_id.notnull()]
df1.count()

In [6]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3091024 entries, 0 to 3091023
Data columns (total 15 columns):
marketplace          object
customer_id          int64
review_id            object
product_id           object
product_parent       int64
product_title        object
product_category     object
star_rating          int64
helpful_votes        int64
total_votes          int64
vine                 object
verified_purchase    object
review_headline      object
review_body          object
review_date          object
dtypes: int64(5), object(10)
memory usage: 353.7+ MB


In [7]:
# print(df[['product_id']].groupby(df['product_id']).count())
# mean of goup by item
print(df1[['product_id']].groupby(df1['product_id']).count().mean())
# print(df[['customer_id']].groupby(df['customer_id']).count())
# mean of goup by user
print(df1[['customer_id']].groupby(df['customer_id']).count().mean())
print(df1[['star_rating']].astype('int').mean())

product_id    16.638625
dtype: float64
customer_id    1.435834
dtype: float64
star_rating    4.035415
dtype: float64


In [8]:
# Check numbers of unique products & customers
print(len(df1['product_id'].unique().tolist()))
print(len(df1['customer_id'].unique().tolist()))

185774
2152773


In [9]:
from collections import defaultdict
from itertools import count

uid_map = defaultdict(count().__next__)
iid_map = defaultdict(count().__next__)
uids = np.array([uid_map[uid] for uid in df1["customer_id"].values ], dtype=np.int32)
iids = np.array([iid_map[iid] for iid in df1["product_id"].values], dtype=np.int32)

uid_rev_map = {v: k for k, v in uid_map.items()}
iid_rev_map = {v: k for k, v in iid_map.items()}

ratings = df1["star_rating"].values.astype(np.float32)
timestamps = df1["review_date"].values.astype(np.object)

print("userId %d got uid %d" % (41409413, uid_map[41409413]))
print("itemId %s got iid %s" % ('B00428R89M', iid_map['B00428R89M']))

userId 41409413 got uid 0
itemId B00428R89M got iid 0


In [10]:
from spotlight.cross_validation import user_based_train_test_split
from spotlight.datasets.synthetic import generate_sequential
from spotlight.evaluation import sequence_mrr_score
from spotlight.sequence.implicit import ImplicitSequenceModel
from spotlight.interactions import Interactions
from spotlight.cross_validation import random_train_test_split
imp_dataset = Interactions(user_ids=uids, item_ids=iids, num_users=len(uid_map), num_items=len(iid_rev_map))

In [11]:
from spotlight.cross_validation import random_train_test_split

itrain, itest = random_train_test_split(imp_dataset, random_state=np.random.seed(42))
print(itrain)
print(itest)

<Interactions dataset (2152773 users x 185774 items x 2472819 interactions)>
<Interactions dataset (2152773 users x 185774 items x 618205 interactions)>


Training - Implicit model with defult parameter

In [12]:
from spotlight.factorization.implicit import ImplicitFactorizationModel
import time  

imodel = ImplicitFactorizationModel(n_iter=10, #I set 3 before
                                    embedding_dim=32, #this is Spotlight default
                                    use_cuda=False)
current = time.time()

imodel.fit(itrain, verbose=True)
end = time.time()
diff = end - current
print("Training took %d seconds" % (diff))

/usr/local/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch 0: loss 0.18269001012185962
Epoch 1: loss 0.0435425656632661
Epoch 2: loss 0.011235846916076729
Training took 48556 seconds


In [13]:
print(imodel.predict(0))
print(len(imodel.predict(0)))

[11.480395    5.2460804  11.873865   ...  0.6930567   0.62018776
 -1.6039956 ]
185774


In [14]:
import scipy.stats as st

def tracksForUser(user, k=4):
    ranks = st.rankdata(-imodel.predict(user))  
    for iid in np.argwhere(ranks <= k):
        print(iid_rev_map[iid.item(0)])

tracksForUser(0)

B003L1ZYYM
B0012S4APK
B0002L5R78
B00316263Y


In [16]:
df1[df1["customer_id"]==uid_rev_map.get(1)].groupby("product_id")["product_id"].count()

product_id
B000068O48    1
B00066HQ50    1
B00522RAGI    1
B005ODHJFM    1
B00IBL3MP0    1
Name: product_id, dtype: int64

Evaluation

In [17]:
def scoreAll(model):
    scoresForAllUsers=[]
    for uid in range(0,model._num_users):
        scoresForAllUsers.append(model.predict(uid))
    return scoresForAllUsers

In [18]:
import scipy.stats as st
#returns a numpy array of RR scores for each user.
#parameter: testInteractions -- a test interaction dataset
#parameter: predictions_for_each_user -- a list of numpy arrays of items scores for each user
#parameter: cutoff - a rank cutoff for the MRR user. Default is 0, which means no cutoff.
#parameter: train - the training data for the recommender - if set, it will not evaluate items that were part of the training data.
def calc_reciprank(testInteractions, predictions_for_each_user, train=None, cutoff=0):

    FLOAT_MAX = np.finfo(np.float32).max
    assert testInteractions.num_users == len(predictions_for_each_user), "mismatch: num users in Interactions %d, num users in predictions was %d" % (testInteractions.num_users,len(predictions_for_each_user))
    assert testInteractions.num_items == len(predictions_for_each_user[0]), "mismatch: num items in Interactions %d, num items in predictions was %d" % (testInteractions.num_items, len(predictions_for_each_user[0]))
  
    rrs = []
  
    if train is not None:
        train = train.tocsr()
    
    i=-1
    for user_id, row in enumerate(testInteractions.tocsr()):
        i=i+1
        if not len(row.indices):
            rrs.append(0)
            continue
      
        predictions = -predictions_for_each_user[i]
    
    #if train is set, remove the interactions from the training dataset
        if train is not None:
            predictions[train[user_id].indices] = FLOAT_MAX
    
        rank = st.rankdata(predictions)[row.indices].min()
    
    #we may have a rank cutoff
        if cutoff > 0 and rank > cutoff:
            rrs.append(0)
            continue
    
        rrs.append(1.0 / rank)
    return np.array(rrs)

ERROR happens since here

In [ ]:
print(calc_reciprank(itest, scoreAll(imodel)).mean())
print(calc_reciprank(itest, scoreAll(imodel), train=itrain).mean())

In [ ]:
rrs = calc_reciprank(itest, scoreAll(imodel))
RR_ranks = st.rankdata(-rrs, method='min')

def countRatings(uid, rr):
    print("User with id %d with RR %f had %d ratings" %( uid, rr, len(iids[uids==uid])))

countRatings( np.argwhere(RR_ranks==1).item(0), rrs[RR_ranks==1].item(0) )

countRatings( np.argwhere(RR_ranks==RR_ranks.max()).item(0), rrs[RR_ranks==RR_ranks.max()].item(0) )